In [4]:
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import load_model
import pickle
import random

# Load the MIDI file
midi_file = converter.parse('example.mid')

# Extract notes and chords from the MIDI file
notes = []
for element in midi_file.flat.notes:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.pitches))

# Save the notes for later use
with open('notes.pkl', 'wb') as filepath:
    pickle.dump(notes, filepath)

# Load the trained model
model = load_model('lstm.keras')

# Prepare the input sequence
sequence_length = 32
n_vocab = len(set(notes))
note_to_int = {note: number for number, note in enumerate(sorted(set(notes)))}

# Function to generate notes with slight variation
def generate_notes_with_variation(model, network_input, pitchnames, n_vocab, variation_percentage=0.25):
    start = np.random.randint(0, len(network_input)-1)
    pattern = network_input[start]
    prediction_output = []
    
    for note_index in range(500):  # Generate 500 notes
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        
        # Ensure the index is within the range of pitchnames
        if index < len(pitchnames):
            result = pitchnames[index]
        else:
            result = pitchnames[-1]  # Default to the last note in case of out-of-range index
        
        # Add variation (25% difference from the original)
        if random.random() < variation_percentage:
            # Apply small random changes to the pitch
            if '.' in result:  # If it's a chord
                notes_in_chord = result.split('.')
                new_notes_in_chord = []
                for note_str in notes_in_chord:
                    pitch = note.Note(note_str)
                    # Apply small random transposition (-1 or +1 semitone)
                    pitch.transpose(random.choice([-1, 1]))
                    new_notes_in_chord.append(str(pitch.pitch))
                result = '.'.join(new_notes_in_chord)
            else:
                pitch = note.Note(result)
                # Apply small random transposition (-1 or +1 semitone)
                pitch.transpose(random.choice([-1, 1]))
                result = str(pitch.pitch)
        
        prediction_output.append(result)
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

# Prepare the input for the model
network_input = []
for i in range(0, len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    network_input.append([note_to_int[note] for note in sequence_in])

# Generate notes with slight variation
pitchnames = sorted(set(notes))
prediction_output = generate_notes_with_variation(model, network_input, pitchnames, n_vocab)

# Create MIDI from the generated notes
def create_midi(prediction_output, output_file='output_variation.mid'):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if '.' in pattern or pattern.isdigit():  # It's a chord
            notes_in_chord = pattern.split('.')
            chord_notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(current_note)
                new_note.storedInstrument = instrument.Piano()
                chord_notes.append(new_note)
            new_chord = chord.Chord(chord_notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            # It's a single note
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5  # Adjust the offset for the next note

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)

# Create and save MIDI file from generated notes
create_midi(prediction_output)
